In [1]:
!pip install langgraph transformers accelerate bitsandbytes --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install huggingface_hub --quiet
print("✅ All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU detected. Please ensure you're using a GPU runtime.")

CUDA available: True
GPU device: Tesla T4
CUDA version: 12.4
GPU memory: 14.74 GB


In [6]:
from huggingface_hub import notebook_login
print("Please login to Hugging Face to access Llama models:")
notebook_login()

Please login to Hugging Face to access Llama models:


In [10]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig
)
import torch, gc
from typing import Dict, List, Optional
from dataclasses import dataclass, field
from datetime import datetime
import textwrap, logging, warnings
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

print("✅ Model loaded successfully!")
print(f"Model device: {model.device}")
print(f"Memory usage: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

Loading model: meta-llama/Llama-3.2-3B-Instruct
This may take a few minutes...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Model loaded successfully!
Model device: cuda:0
Memory usage: 2.09 GB


In [11]:
class EnhancedTutorSystem:
    """Enhanced tutor system with Llama-3.2-3B-Instruct"""

    def __init__(self, model_pipe, max_retries=3):
        self.pipe = model_pipe
        self.max_retries = max_retries

    def format_llama_prompt(self, system_msg: str, user_msg: str) -> str:
        """Format prompt for Llama-3.2-Instruct"""
        return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    def query_model_with_retry(self, prompt: str, system_message: str = "", max_tokens: int = 200) -> str:
        """Query model with retry logic and error handling"""
        full_prompt = self.format_llama_prompt(system_message, prompt)

        for attempt in range(self.max_retries):
            try:
                result = self.pipe(
                    full_prompt,
                    max_new_tokens=max_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    pad_token_id=self.pipe.tokenizer.eos_token_id
                )

                response = result[0]["generated_text"]

                if "<|start_header_id|>assistant<|end_header_id|>" in response:
                    response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
                    response = response.replace("<|eot_id|>", "").strip()

                return response

            except Exception as e:
                logger.warning(f"Model query attempt {attempt + 1} failed: {e}")
                if attempt == self.max_retries - 1:
                    return f"Error generating response: {str(e)}"

        return "Unable to generate response after multiple attempts."

    def assessment_agent(self, state: Dict) -> Dict:
        """Generate assessment questions with difficulty adaptation"""
        difficulty_prompts = {
            "easy": "Ask a basic, fundamental question that tests basic understanding",
            "medium": "Ask a moderately challenging question that requires some analysis",
            "hard": "Ask an advanced, complex question that requires deep thinking"
        }

        history_context = ""
        if state.get("question_history"):
            recent_questions = state["question_history"][-3:]
            history_context = f"\nAvoid these recent questions: {', '.join(recent_questions)}"

        prompt = f"""
Topic: {state['topic']}
Difficulty: {state.get('difficulty_level', 'medium')}

{difficulty_prompts.get(state.get('difficulty_level', 'medium'), difficulty_prompts['medium'])} about {state['topic']}.
Make it educational, clear, and engaging.{history_context}

Provide only the question, nothing else.
"""
        system = "You are an expert tutor. Create clear, educational questions that test understanding. Respond with only the question."

        question = self.query_model_with_retry(prompt, system)
        question = question.strip()
        if question.startswith("Question:"):
            question = question[9:].strip()

        return {"question": question}

    def explainer_agent(self, state: Dict) -> Dict:
        """Provide detailed feedback with structured analysis"""
        prompt = f"""
Question: {state['question']}
Student Answer: {state['user_answer']}

Analyze the student's answer and provide feedback in this format:

CORRECTNESS: [Correct/Partially Correct/Incorrect]

ANALYSIS: [Explain what's right or wrong about the answer]

CORRECT ANSWER: [Provide the complete correct answer]

IMPROVEMENT TIPS: [How the student can improve]
"""
        system = "You are a patient, detailed tutor. Provide structured, encouraging feedback that helps students learn."
        feedback = self.query_model_with_retry(prompt, system, max_tokens=300)

        return {"feedback": feedback}

    def quiz_agent(self, state: Dict) -> Dict:
        """Generate practice questions with variety"""
        prompt = f"""
Topic: {state['topic']}
Previous question: {state['question']}

Create a different type of practice question about {state['topic']}.
Make it engaging, educational, and different from the previous question.

Provide only the practice question, nothing else.
"""
        system = "You are a creative tutor. Generate diverse, engaging practice questions. Respond with only the question."
        quiz = self.query_model_with_retry(prompt, system)

        quiz = quiz.strip()
        if quiz.startswith("Practice Question:"):
            quiz = quiz[17:].strip()

        return {"quiz": quiz}

    def progress_agent(self, state: Dict) -> Dict:
        """Enhanced progress tracking with difficulty adaptation"""
        feedback_lower = state.get("feedback", "").lower()

        correct_indicators = ["correct", "right", "good", "excellent", "well done"]
        incorrect_indicators = ["incorrect", "wrong", "not quite", "partially"]

        is_correct = any(ind in feedback_lower for ind in correct_indicators)
        is_incorrect = any(ind in feedback_lower for ind in incorrect_indicators)

        if "partially" in feedback_lower and "correct" in feedback_lower:
            is_correct = True
        elif is_incorrect:
            is_correct = False

        performance_history = state.get("performance_history", [])
        performance_history.append(is_correct)

        new_score = state.get("score", 0) + (1 if is_correct else 0)
        total_questions = state.get("total_questions", 0) + 1

        current_difficulty = state.get("difficulty_level", "medium")
        new_difficulty = current_difficulty

        if len(performance_history) >= 3:
            recent_performance = sum(performance_history[-3:]) / 3
            if recent_performance >= 0.8 and current_difficulty != "hard":
                new_difficulty = "hard" if current_difficulty == "medium" else "medium"
            elif recent_performance <= 0.4 and current_difficulty != "easy":
                new_difficulty = "easy" if current_difficulty == "medium" else "medium"

        return {
            "score": new_score,
            "total_questions": total_questions,
            "difficulty_level": new_difficulty,
            "performance_history": performance_history,
            "is_correct": is_correct
        }

    def get_performance_summary(self, state: Dict) -> str:
        """Generate a performance summary"""
        total_questions = state.get("total_questions", 0)
        if total_questions == 0:
            return "No questions answered yet."

        score = state.get("score", 0)
        accuracy = (score / total_questions) * 100
        session_start = state.get("session_start", datetime.now())
        duration = datetime.now() - session_start

        summary = f"""
📊 Performance Summary:
• Questions Answered: {total_questions}
• Correct Answers: {score}
• Accuracy: {accuracy:.1f}%
• Current Difficulty: {state.get('difficulty_level', 'medium').title()}
• Session Duration: {str(duration).split('.')[0]}
"""
        return summary.strip()

print("✅ Enhanced Tutor System class defined!")


✅ Enhanced Tutor System class defined!


In [12]:
from langgraph.graph import StateGraph

tutor_system = EnhancedTutorSystem(pipe)
workflow = StateGraph(dict)

workflow.add_node("assess", tutor_system.assessment_agent)
workflow.add_node("explain", tutor_system.explainer_agent)
workflow.add_node("quiz", tutor_system.quiz_agent)
workflow.add_node("progress", tutor_system.progress_agent)

workflow.add_edge("assess", "explain")
workflow.add_edge("explain", "quiz")
workflow.add_edge("quiz", "progress")
workflow.add_edge("progress", "assess")

workflow.set_entry_point("assess")
app = workflow.compile()

print("✅ LangGraph workflow compiled successfully!")

✅ LangGraph workflow compiled successfully!


In [15]:
def run_interactive_tutor_with_input():
    """
    Prompt user for topic and number of rounds, then run a full tutoring session.
    """
    from IPython.display import display, Markdown

    # Ask user for topic
    print("📚 Available Topics: algebra, data structures, biology, operating systems, python programming, computer networks, chemistry")
    topic = input("🔍 Enter the topic you want to study: ").strip().lower()
    if not topic:
        topic = "data structures"

    # Ask for number of rounds
    try:
        num_rounds = int(input("🔢 Enter number of questions/rounds (default 3): "))
    except:
        num_rounds = 3

    # Initialize session state
    state = {
        "topic": topic,
        "difficulty_level": "medium",
        "score": 0,
        "total_questions": 0,
        "question_history": [],
        "performance_history": [],
        "session_start": datetime.now()
    }

    print("=" * 60)
    print("🎓 Enhanced AI Tutor Interactive Session")
    print("=" * 60)
    print(f"📘 Topic: {topic.title()}")
    print(f"🔁 Number of Rounds: {num_rounds}")

    for round_num in range(num_rounds):
        print(f"\n🧠 Round {round_num + 1} / {num_rounds}")
        print("-" * 50)

        # Step 1: Assessment agent - update state
        assess_output = tutor_system.assessment_agent(state)
        state.update(assess_output)

        # Show the generated question
        display(Markdown(f"### 📘 Question ({state['difficulty_level'].title()}):\n{state['question']}"))
        state["question_history"].append(state["question"])

        # Step 2: Get user answer
        answer = input("👤 Your answer: ").strip()
        if not answer:
            answer = "I don't know"
        state["user_answer"] = answer

        # Step 3: Run explainer
        print("\n🤖 Analyzing your answer...")
        feedback_output = tutor_system.explainer_agent(state)
        state.update(feedback_output)
        display(Markdown(f"### 📝 Feedback:\n{state['feedback']}"))

        # Step 4: Run quiz generator
        print("\n🎲 Generating a practice question...")
        quiz_output = tutor_system.quiz_agent(state)
        state.update(quiz_output)
        display(Markdown(f"### 🧪 Practice Question:\n{state['quiz']}"))
        state["question_history"].append(state["quiz"])

        # Step 5: Run progress tracker
        progress_output = tutor_system.progress_agent(state)
        state.update(progress_output)

        # Show progress update
        print(f"\n📊 Score: {state['score']} / {state['total_questions']}")
        print("✅ Correct!" if state.get("is_correct") else "❌ Incorrect, try again!")
        print(f"🔄 Difficulty Level: {state['difficulty_level'].title()}")

        if round_num < num_rounds - 1:
            input("\n▶️ Press Enter to continue to next round...")

    # Final session summary
    print("\n🎉 Session Complete!")
    print("=" * 60)
    display(Markdown(f"## 📊 Final Performance Summary\n\n{tutor_system.get_performance_summary(state)}"))

    return state

# 🚀 Run the interactive session
run_interactive_tutor_with_input()

📚 Available Topics: algebra, data structures, biology, operating systems, python programming, computer networks, chemistry
🔍 Enter the topic you want to study: biology
🔢 Enter number of questions/rounds (default 3): 2
🎓 Enhanced AI Tutor Interactive Session
📘 Topic: Biology
🔁 Number of Rounds: 2

🧠 Round 1 / 2
--------------------------------------------------


### 📘 Question (Medium):
What is the primary function of the chloroplasts in plant cells, and how do they contribute to the process of photosynthesis?

👤 Your answer: Convert light energy from the sun into chemical energy in the form of glucose

🤖 Analyzing your answer...


### 📝 Feedback:
Here's my analysis and feedback for the student's answer:

CORRECTNESS: Partially Correct

ANALYSIS:
The student correctly identified that chloroplasts play a key role in converting light energy from the sun into chemical energy. However, their answer is incomplete and lacks specificity. The student failed to mention the specific products of photosynthesis (e.g., oxygen, water, and ATP) and the overall equation for photosynthesis.

CORRECT ANSWER:
The primary function of chloroplasts in plant cells is to convert light energy from the sun into chemical energy through the process of photosynthesis. This process involves the conversion of carbon dioxide and water into glucose and oxygen using sunlight as an energy source. The overall equation for photosynthesis is:

6 CO2 + 6 H2O + light energy → C6H12O6 (glucose) + 6 O2

IMPROVEMENT TIPS:

* Make sure to include all the key components involved in photosynthesis, such as the reactants (carbon dioxide and water), products (glucose and oxygen), and energy source (light).
* Be more precise with your terminology - instead of just saying "convert light energy," consider specifying the type of light energy (e.g., visible light, ultraviolet light).
* Practice writing out the full equation for photosynthesis to ensure you understand the entire process.
* Double-check your answers by reviewing the molecular structure of glucose and the role of chlorophyll in absorbing light energy!


🎲 Generating a practice question...


...


🎲 Generating a practice question...


### 🧪 Practice Question:
How does the process of horizontal gene transfer, where genetic material is transferred between organisms other than by vertical inheritance (parent-to-offspring), contribute to the evolution of antibiotic resistance in bacteria?


📊 Score: 2 / 2
✅ Correct!
🔄 Difficulty Level: Medium

🎉 Session Complete!


## 📊 Final Performance Summary

📊 Performance Summary:
• Questions Answered: 2
• Correct Answers: 2
• Accuracy: 100.0%
• Current Difficulty: Medium
• Session Duration: 0:02:31

{'topic': 'biology',
 'difficulty_level': 'medium',
 'score': 2,
 'total_questions': 2,
 'question_history': ['What is the primary function of the chloroplasts in plant cells, and how do they contribute to the process of photosynthesis?',
  'What is the unique characteristic that allows certain species of jellyfish to be immortal, and what biological processes underlie this phenomenon?',
  'What is the role of epigenetic modifications in regulating gene expression in response to environmental stimuli, and how do these changes impact the development and adaptation of organisms?',
  'How does the process of horizontal gene transfer, where genetic material is transferred between organisms other than by vertical inheritance (parent-to-offspring), contribute to the evolution of antibiotic resistance in bacteria?'],
 'performance_history': [True, True],
 'session_start': datetime.datetime(2025, 7, 11, 9, 42, 41, 164232),
 'question': 'What is the role of epigenetic modifications in regulatin